In [0]:
import torch
torch.cuda.get_device_name(0)
# 'Tesla P100-PCIE-16GB' worked great for me

'Tesla P100-PCIE-16GB'

In [0]:
"""
INSTRUCTIONS:
- upload data.py, util.py, preprocessing.py (optional) in ./content and edit them in a way relative paths work(when importing)
- i managed to build an engine by creating corpus(DataFrame) of abstracts back in the python console localy, then serialize it 
  with pickle, upload it here in /content and then use it here along with paper_ids in query_engine.fit method
- the rest is basically executing the code given here with possible minor tweaks
"""

In [0]:
!pip install sentence-transformers

     |████████████████████████████████| 61kB 3.2MB/s 
     |████████████████████████████████| 573kB 13.9MB/s 
     |████████████████████████████████| 3.7MB 57.6MB/s 
     |████████████████████████████████| 890kB 57.4MB/s 
     |████████████████████████████████| 1.0MB 46.2MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-cp36-none-any.whl size=74031 sha256=202b31ce2c0702d11be4ac0ad402f02184312a8ab16833bc0315b28b7c346ea2
  Stored in directory: /root/.cache/pip/wheels/d7/fa/17/2b081a8cd8b0a86753fb0e9826b3cc19f0207062c0b2da7008
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=65e5a5a8e72511c7b7a8d3e14e546079b267272ad7b41d729c6f4d7f7ebaf5c5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [0]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import models
from nltk import sent_tokenize
import numpy as np

In [0]:
def normalize(embeddings):
        """
        Normalizes embeddings using L2 normalization.
        Args:
            embeddings: input embeddings matrix
        Returns:
            normalized embeddings
        """
        # Calculation is different for matrices vs vectors
        if len(embeddings.shape) > 1:
            return embeddings / np.linalg.norm(embeddings, axis=1).reshape(-1, 1)

        else:
            return embeddings / np.linalg.norm(embeddings)

In [0]:
def BERT_sentence_embeddings(data, query=False):
    
    """
    Input:
        corpus: DataFrame containing information about paragraphs : paper_id, section, text
        query: if True, import is one sentence - a query
    Returns:
        corpus embeddings: numpy array containing paragraph embeddings for each text paragraph in input
        which is obtained by averaging over sentence embeddings(try #1 - until a better idea arrives (probably not so great))
        -dimensions: n x 768 where n represents number of input paragraphs
    
    References
    ----------
    {
    reimers-2019-sentence-bert,
    title = "Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks",
    author = "Reimers, Nils and Gurevych, Iryna",
    booktitle = "Proceedings of the 2019 Conference on Empirical Methods in Natural Language Processing",
    month = "11",
    year = "2019",
    publisher = "Association for Computational Linguistics",
    url = "http://arxiv.org/abs/1908.10084",
    }

    """
    
    #pre-trained model on semantic text similarity task
    model = SentenceTransformer('bert-base-nli-stsb-mean-tokens') 
    
    if query:
        return normalize(np.array(model.encode([data])).reshape(1,768))
    
    else:
        text_paragraphs = [paragraph for paragraph in list(data['text'])]
        n=len(text_paragraphs)
        
        corpus_embeddings=[]
        for paragraph in text_paragraphs:
            sentences = sent_tokenize(paragraph)
            sent_embeddings = normalize(np.array(model.encode(sentences)).reshape(-1,768))#shape = no_of_sents_in_paragraph X 768
            corpus_embeddings.append(np.mean(sent_embeddings,axis=0).reshape(1,768)) 
        
        return normalize(np.array(corpus_embeddings).reshape(n,768))

In [0]:
emb1 = BERT_sentence_embeddings('What do we know about COVID-19 risk factors?', query=True)
emb2 = BERT_sentence_embeddings('Do co-existing respiratory/viral infections make the virus more transmissible or virulent?', query=True)
emb3 = BERT_sentence_embeddings('The text is small and will load quickly and easily fit into memory.', query=True)

100%|██████████| 405M/405M [00:24<00:00, 16.3MB/s]


In [0]:
#testing sentence embeddings
np.dot(emb1,emb2.T)

array([[0.32969183]], dtype=float32)

In [0]:
np.dot(emb1,emb3.T)

array([[-0.01860096]], dtype=float32)

In [0]:
np.dot(emb3,emb2.T)

array([[0.11308034]], dtype=float32)

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from data import CovidDataLoader
from nltk import sent_tokenize

import pickle
import pandas as pd
import numpy as np
import time

In [0]:
class QueryEngine_BERT():

    def __init__(self):
        self.corpus = None
        self.ids = None

    def fit(self, corpus, document_ids=None):
        """
        Builds the query engine on the given corpus.

        Args:
            corpus: list of documents to build the model on
            document_ids: optional, if given it will associate the given id's to each document given in the corpus
        """
        self.ids = document_ids
        self.corpus = [paragraph for paragraph in corpus['text']]
        self.corpus_embeddings = BERT_sentence_embeddings(corpus, query=False)
        
    def __create_query_result(self, query, similarities, n):
        """

        Args:
            similarities: sparse matrix containing cosine similarities between the query vector and documents from corpus
            n: number of most similar documents to include in the result

        Returns:
            pandas DataFrame containing query, document, similarity
        """

        result = {
            'query': query * len(similarities),
            'text': self.corpus,
            'sim': np.squeeze(similarities)
        }
        if self.ids:
            result.update({'id': self.ids})

        result = pd.DataFrame(result).sort_values(by='sim', ascending=False)[:n]

        return result[result['sim'] > 0]

    def run_query(self, query, n=5):
        """
        Runs the given query, returns max n most similar documents from the corpus on which the model was build.

        Args:
            query: query to run
            n: max number of results returned

        Returns:
            n(or less) most similar documents from the corpus on which the model was build
        """
        if self.corpus is None:
            raise AttributeError('Model not built yet, please call the fit method before running queries!')

        assert type(query) == str
            

        query_embedding = BERT_sentence_embeddings(query, query=True)
        similarities = np.dot(self.corpus_embeddings,query_embedding.T)  # TODO: check if this already sorts values
        
        return self.__create_query_result(query, similarities, n)

    def save(self, dir_path, name):
        """
        Serializes the object to file(name.dat) to the directory defined by the path.

        Args:
            dir_path: path of the directory to save the object to
            name: name of the file without any extensions
        """
        pickle_path = dir_path + name + '.dat'
        print('Writing object to %s' % pickle_path)
        with open(pickle_path, 'wb') as f:
            pickle.dump(self, f)

    @staticmethod
    def load(pickle_path):
        """
        Loads(de-serializes) QueryEngine object from the given path.

        Args:
            pickle_path: path to QueryEngine pickle

        Returns:
            QueryEngine object
        """
        with open(pickle_path, 'rb') as f:
            query_engine = pickle.load(f)
            if type(query_engine) != QueryEngine_BERT:
                raise ValueError('Path to non QueryEngine_BERT object!')
            return query_engine

In [0]:
with open("test.txt", "rb") as fp:
  abstracts = pickle.load(fp)

In [0]:
abstracts

[{'paper_id': '0015023cc06b5362d332b3baf348d11567ca2fbb',
  'section': 'Abstract',
  'text': 'word count: 194 22 Text word count: 5168 23 24 25 author/funder. All rights reserved. No reuse allowed without permission. Abstract 27 The positive stranded RNA genomes of picornaviruses comprise a single large open reading 28 frame flanked by 5′ and 3′ untranslated regions (UTRs). Foot-and-mouth disease virus (FMDV) 29 has an unusually large 5′ UTR (1.3 kb) containing five structural domains. These include the 30 internal ribosome entry site (IRES), which facilitates initiation of translation, and the cis-acting 31 replication element (cre). Less well characterised structures are a 5′ terminal 360 nucleotide 32 stem-loop, a variable length poly-C-tract of approximately 100-200 nucleotides and a series of 33 two to four tandemly repeated pseudoknots (PKs). We investigated the structures of the PKs 34 by selective 2′ hydroxyl acetylation analysed by primer extension (SHAPE) analysis and 35 dete

In [0]:
len(abstracts)

29306

In [0]:
paper_ids = [abstract['paper_id'] for abstract in abstracts]

In [0]:
query_engine = QueryEngine_BERT()
query_engine.fit(abstracts, paper_ids)
    
query_engine.save('./', 'abstracts_query_engine')

Writing object to ./abstracts_query_engine.dat


In [0]:
query_engine = QueryEngine_BERT.load('abstracts_query_engine.dat')

In [0]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option("display.max_colwidth",None)

In [0]:
query_engine.run_query('Physical science of the coronavirus',10)[['text','sim']].style.set_properties(**{'font-size': '7pt'})

,text,sim
25204,"Coronaviruses (CoVs), enveloped positive-sense RNA viruses, are characterized by club-like spikes that project from their surface, an unusually large RNA genome, and a unique replication strategy. Coronaviruses cause a variety of diseases in mammals and birds ranging from enteritis in cows and pigs and upper respiratory disease in chickens to potentially lethal human respiratory infections. Here we provide a brief introduction to coronaviruses discussing their replication and pathogenicity, and current prevention and treatment strategies. We also discuss the outbreaks of the highly pathogenic Severe Acute Respiratory Syndrome Coronavirus (SARS-CoV) and the recently identifi ed Middle Eastern Respiratory Syndrome Coronavirus (MERS-CoV). Coronavirus virions are spherical with diameters of approximately 125 nm as depicted in recent studies by cryo-electron tomography and cryo-electron microscopy [ 2 , 3 ]. The most prominent feature of coronaviruses is the club-shaped spike projections emanating from the surface of the virion. These spikes are a defi ning feature of the virion and give them the appearance of a solar corona, prompting the name, coronaviruses. Within the envelope of the virion is the nucleocapsid. Coronaviruses have helically symmetrical nucleocapsids, which is uncommon among positive-sense RNA viruses, but far more common for negative-sense RNA viruses.",0.631149
12368,"Coronaviruses are etiologic agents of respiratory and enteric diseases in humans and in animals. In this study, a one-step real-time reverse transcriptionpolymerase chain reaction (RT-PCR) assay based on SYBR Green chemistry and degenerate primers was developed for the generic detection of coronaviruses. The primers, designed in the open reading frame 1b, enabled the detection of 32 animal coronaviruses including strains of canine coronavirus, feline coronavirus, transmissible gastroenteritis virus (TGEV), bovine coronavirus (BCoV), murine hepatitis virus (MHV) and infectious bronchitis virus (IBV). A specific amplification was also observed with the human coronaviruses (HCoV) HCoV-NL63, HCoV-OC43, HCoV-229E and severe acute respiratory syndrome coronavirus (SARS-CoV). The real-time RT-PCR detected down to 10 cRNA copies from TGEV, BCoV, SARS-CoV and IBV. In addition, the assay exhibited a high sensitivity and specificity on clinical samples from different animal species. The developed assay represents a potential tool for laboratory diagnostics and for detecting still uncharacterized coronaviruses.",0.611510
4371,"In 2012, a novel coronavirus, initially named as human coronavirus EMC (HCoV-EMC) but recently renamed as Middle East respiratory syndrome human coronavirus (MERS-CoV), was identified in patients who suffered severe acute respiratory infection and subsequent renal failure that resulted in death. Ongoing epidemiological investigations together with retrospective studies have found 61 laboratory-confirmed cases of infection with this novel coronavirus, including 34 deaths to date. This novel coronavirus is culturable and two complete genome sequences are now available. Furthermore, molecular detection and indirect immunofluorescence assay have been developed. The present paper summarises the limited recent advances of this novel human coronavirus, including its discovery, genomic characterisation and detection. HCoV-EMC, MERS-CoV, genomic characterisation, molecular detection Geng H Y, Tan W J. A novel human coronavirus: Middle East respiratory syndrome human coronavirus.",0.609226
17116,"Severe acute respiratory syndrome (SARS) was first described during a 2002-2003 global outbreak of severe pneumonia associated with human deaths and person-toperson disease transmission. The etiologic agent was initially identified as a coronavirus by thin-section electron microscopic examination of a virus isolate. Virions were spherical, 78 nm in mean diameter, and composed of a helical nucleocapsid within an envelope with surface proje

In [0]:
"""
TODO: analogno za body_texts
"""